In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
df = pd.read_csv(r"../raw_data/data_house_full/companydata_part1.csv", dtype=str, low_memory=True)


In [3]:
uk_countries = ["ENGLAND", "SCOTLAND", "WALES", "NORTHERN IRELAND", "UNITED KINGDOM"]

In [4]:
df_uk = df[df["RegAddress.Country"].isin(uk_countries)]

In [5]:
df_uk.head()

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
0,! LTD,08209948,NaN,NaN,9 PRINCES SQUARE,NaN,HARROGATE,NaN,ENGLAND,HG1 1ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2025,11/09/2024
1,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29/12/2025,15/12/2024
2,!NFLECTION ADVISORY LIMITED,15073164,NaN,NaN,52 MEADWAY,NaN,BARNET,NaN,UNITED KINGDOM,EN5 5LB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28/08/2025,14/08/2024
3,!NFOGENIE LTD,13522064,NaN,NaN,71-75 SHELTON STREET,NaN,LONDON,GREATER LONDON,UNITED KINGDOM,WC2H 9JQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/08/2025,20/07/2024
4,!NK MEDIA LIMITED,15801429,NaN,NaN,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON ...",FIRST FLOOR,LONDON,NaN,ENGLAND,W1W 7LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16/06/2026,02/06/2025


In [7]:
df_uk.columns = df_uk.columns.str.strip()

In [8]:
# 2) Clean address + postcode
def clean_str(s): return s.fillna("").str.strip().str.upper()
def clean_postcode(s):
    pc = clean_str(s).str.replace(r"[^A-Z0-9]", "", regex=True)
    return np.where(pc.str.len() < 5, pc, pc.str[:-3] + " " + pc.str[-3:])

In [9]:
df_uk["AddressLine1"]   = clean_str(df_uk["RegAddress.AddressLine1"])
df_uk["PostCode_clean"] = clean_postcode(df_uk["RegAddress.PostCode"])

/tmp/ipykernel_163412/564625837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine1"]   = clean_str(df_uk["RegAddress.AddressLine1"])
/tmp/ipykernel_163412/564625837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["PostCode_clean"] = clean_postcode(df_uk["RegAddress.PostCode"])


In [10]:
df_uk["AddressLine2"]   = clean_str(df_uk["RegAddress.AddressLine2"])

/tmp/ipykernel_163412/713273805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine2"]   = clean_str(df_uk["RegAddress.AddressLine2"])


In [11]:
df_uk["Town_clean"]   = clean_str(df_uk.get("RegAddress.PostTown", pd.Series("", index=df_uk.index)))

/tmp/ipykernel_163412/1228941311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Town_clean"]   = clean_str(df_uk.get("RegAddress.PostTown", pd.Series("", index=df_uk.index)))


In [12]:
# 2) Heuristic: choose the street-like line
#    - If AddressLine1 has street keywords, keep it.
#    - Else if AddressLine2 has them, use AddressLine2.
#    - Else fall back to AddressLine1.
STREET_WORDS = r"\b(ROAD|RD|STREET|ST|AVENUE|AVE|LANE|LN|DRIVE|DR|COURT|CT|WAY|PLACE|PL|SQUARE|SQ|CRESCENT|CRES|TERRACE|TER|GROVE|PARADE|HILL|HIGH\s+ST(REET)?|HIGH\s+RD|CLOSE|MEWS|ROW|QUAY|QUAYS|GATE|MARKET|MARKET\s+PLACE)\b"

l1_is_street = df_uk["AddressLine1"].str.contains(STREET_WORDS, regex=True, na=False)
l2_is_street = df_uk["AddressLine2"].str.contains(STREET_WORDS, regex=True, na=False)

df_uk["Address_street"] = np.where(
    l1_is_street, df_uk["AddressLine1"],
    np.where(l2_is_street, df_uk["AddressLine2"], df_uk["AddressLine1"])
)

# (Optional) build a display-friendly full address
df_uk["FullAddress"] = (
    df_uk["AddressLine1"]
      .str.cat(df_uk["AddressLine2"].mask(df_uk["AddressLine2"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
      .str.cat(df_uk["Town_clean"].mask(df_uk["Town_clean"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
      .str.cat(df_uk["PostCode_clean"].mask(df_uk["PostCode_clean"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
)


/tmp/ipykernel_163412/1380917566.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l1_is_street = df_uk["AddressLine1"].str.contains(STREET_WORDS, regex=True, na=False)
/tmp/ipykernel_163412/1380917566.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l2_is_street = df_uk["AddressLine2"].str.contains(STREET_WORDS, regex=True, na=False)
/tmp/ipykernel_163412/1380917566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street"] = np.where(
/tmp/ipykernel_163412/1380917566.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [13]:
# 3) Count unique companies per (postcode, chosen street address)
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

In [14]:
address_counts = (
    df_uk.groupby(["PostCode_clean", "Address_street"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
         .sort_values("Companies_at_Address", ascending=False)
)

In [15]:
# 4) Postcode totals (for share)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

In [16]:
# 5) Merge + share; show heavy hitters (>=100)
compare = address_counts.merge(postcode_counts, on="PostCode_clean", how="left")
compare["Address_share_in_PC"] = (compare["Companies_at_Address"] / compare["Companies_in_Postcode"] * 100).round(1)

heavy_100plus = compare[compare["Companies_at_Address"] >= 100].copy() \
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)

heavy_100plus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188 entries, 0 to 184
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         188 non-null    object 
 1   Address_street         188 non-null    object 
 2   Companies_at_Address   188 non-null    int64  
 3   Companies_in_Postcode  188 non-null    int64  
 4   Address_share_in_PC    188 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.8+ KB


In [17]:
heavy_100plus.head()

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC
0,WC2H 9JQ,71-75 SHELTON STREET,11883,12062,98.5
1,EC1V 2NX,128 CITY ROAD,5806,9248,62.8
2,N1 7GU,20-22 WENLOCK ROAD,2710,5408,50.1
3,N1 7GU,20 WENLOCK ROAD,2623,5408,48.5
4,W1W 7LT,85 GREAT PORTLAND STREET,2409,2843,84.7


In [18]:
# 1) Build a one-row-per-(postcode, street) map with a single chosen FullAddress
def best_full(series: pd.Series) -> str:
    s = series.replace("", np.nan).dropna()
    if s.empty:
        return np.nan
    vc = s.value_counts()
    top_n = vc[vc.eq(vc.iloc[0])].index.tolist()  # all tied for most frequent
    # tie-breaker: prefer the one with more components (more commas), then longer length
    return max(top_n, key=lambda a: (a.count(","), len(a)))

fa_map = (
    df_uk.groupby(["PostCode_clean", "Address_street"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress": "FullAddress_best"})
)

# 2) Left-merge onto your heavy_100plus WITHOUT creating extra rows
heavy_100plus = heavy_100plus.merge(
    fa_map,
    on=["PostCode_clean", "Address_street"],
    how="left",
    validate="one_to_one"   # guarantees no row multiplication
)


In [19]:
# 3) Fallback if any are missing
heavy_100plus["FullAddress_best"] = heavy_100plus["FullAddress_best"].fillna(
    heavy_100plus["Address_street"] + ", " + heavy_100plus["PostCode_clean"]
)

heavy_100plus.head(10)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress_best
0,WC2H 9JQ,71-75 SHELTON STREET,11883,12062,98.5,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W..."
1,EC1V 2NX,128 CITY ROAD,5806,9248,62.8,"128 CITY ROAD, LONDON, EC1V 2NX"
2,N1 7GU,20-22 WENLOCK ROAD,2710,5408,50.1,"20-22 WENLOCK ROAD, LONDON, N1 7GU"
3,N1 7GU,20 WENLOCK ROAD,2623,5408,48.5,"20 WENLOCK ROAD, LONDON, N1 7GU"
4,W1W 7LT,85 GREAT PORTLAND STREET,2409,2843,84.7,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,..."
5,WC1N 3AX,27 OLD GLOUCESTER STREET,2269,2316,98.0,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX"
6,EC1V 2NX,124 CITY ROAD,2069,9248,22.4,"124 CITY ROAD, LONDON, EC1V 2NX"
7,EC2A 4NE,PAUL STREET,1953,4041,48.3,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE"
8,EC2A 4NE,86-90 PAUL STREET,1806,4041,44.7,"86-90 PAUL STREET, LONDON, EC2A 4NE"
9,W1W 5PF,167-169 GREAT PORTLAND STREET,1322,1786,74.0,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND..."


In [20]:
heavy_100plus


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress_best
0,WC2H 9JQ,71-75 SHELTON STREET,11883,12062,98.5,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W..."
1,EC1V 2NX,128 CITY ROAD,5806,9248,62.8,"128 CITY ROAD, LONDON, EC1V 2NX"
2,N1 7GU,20-22 WENLOCK ROAD,2710,5408,50.1,"20-22 WENLOCK ROAD, LONDON, N1 7GU"
3,N1 7GU,20 WENLOCK ROAD,2623,5408,48.5,"20 WENLOCK ROAD, LONDON, N1 7GU"
4,W1W 7LT,85 GREAT PORTLAND STREET,2409,2843,84.7,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,..."
...,...,...,...,...,...,...
183,B90 8AH,LUMANERI HOUSE BLYTHE GATE,101,116,87.1,"LUMANERI HOUSE BLYTHE GATE, BLYTHE VALLEY PARK..."
184,EC3V 0XL,70 GRACECHURCH STREET,101,102,99.0,"5TH FLOOR, 70 GRACECHURCH STREET, LONDON, EC3V..."
185,SG14 1BP,BELL LANE,101,102,99.0,"BELFRY HOUSE, BELL LANE, HERTFORD, SG14 1BP"
186,SS9 1PE,57A BROADWAY,101,101,100.0,"57A BROADWAY, LEIGH-ON-SEA, SS9 1PE"


In [21]:
# Shows addresses in the same postcode that look similar (ignoring numbers/ranges)
def candidate_groups(heavy_tbl, min_companies=100, postcode=None):
    df = heavy_tbl[heavy_tbl["Companies_at_Address"] >= min_companies].copy()
    if postcode:
        df = df[df["PostCode_clean"].eq(postcode)]
    base = (df["Address_street"]
              .str.replace(r"\b\d+(?:-\d+)?\b", "", regex=True)
              .str.replace(r"[^\w\s]", "", regex=True)
              .str.replace(r"\s+", " ", regex=True)
              .str.strip())
    cand = df.assign(_base=base).groupby(["PostCode_clean","_base"]) \
               .filter(lambda g: g["Address_street"].nunique()>1)
    return cand.sort_values(["PostCode_clean","_base","Companies_at_Address"], ascending=[True,True,False])

# Example:
# candidate_groups(heavy_100plus, postcode="N1 7GU").head(20)


In [22]:
import re
import numpy as np
import pandas as pd

# Normalizer for comparing streets: drop number/ranges, unit/floor words, punctuation, extra spaces
def _norm_street(s: pd.Series) -> pd.Series:
    s = s.fillna("").str.upper()
    s = s.str.replace(r"\b\d+(?:\s*-\s*\d+)?\b", "", regex=True)  # remove numbers & ranges
    s = s.str.replace(r"\b(FLAT|UNIT|SUITE|APT|APARTMENT|ROOM|LEVEL|FLOOR|OFFICE)\b", "", regex=True)
    s = s.str.replace(r"[^\w\s]", "", regex=True)                 # punctuation
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

postcodes_with_issue = []

for pc, sub in heavy_100plus.groupby("PostCode_clean"):
    base = _norm_street(sub["Address_street"])
    # if the normalized base repeats across different Address_street values -> potential merge needed
    dup_mask = base.duplicated(keep=False)
    if dup_mask.any():
        postcodes_with_issue.append(pc)

# Unique + sorted list
postcodes_with_issue = sorted(set(postcodes_with_issue))

# (optional) quick peek
len(postcodes_with_issue), postcodes_with_issue[:10]


(5, ['EC1V 2NX', 'EC2A 4NE', 'HA4 7AE', 'IP28 7DE', 'N1 7GU'])

In [23]:
MERGES = [
    ("N1 7GU", "20 WENLOCK ROAD", "20-22 WENLOCK ROAD")
    # add more lines later if you want
]

In [24]:
# make a working column (keeps your originals intact)
df_uk["Address_street_merged"] = df_uk["Address_street"]

for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_163412/3152388706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [25]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean")[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)
# one representative FullAddress per merged group
fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.dropna().iloc[0] if s.notna().any() else None)
         .reset_index(name="FullAddress_best")
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100).round(1)

heavy_100plus_merged = (heavy_m[heavy_m["Companies_at_Address"] >= 100]
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
                        .rename(columns={"Address_street_merged":"Address_street"}))

# See the result for N1 7GU
heavy_100plus_merged.query('PostCode_clean == "N1 7GU"')


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
217131,N1 7GU,20-22 WENLOCK ROAD,5333,5408,"20-22 WENLOCK ROAD, LONDON, N1 7GU",98.6


In [26]:
# See the result for EC1V 2NX
heavy_100plus_merged.query('PostCode_clean == "EC1V 2NX"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115386,EC1V 2NX,128 CITY ROAD,5806,9248,"128 CITY ROAD, LONDON, EC1V 2NX",62.8
115354,EC1V 2NX,124 CITY ROAD,2069,9248,"124 CITY ROAD, LONDON, EC1V 2NX",22.4
115381,EC1V 2NX,124-128 CITY ROAD,521,9248,"124-128 CITY ROAD, LONDON, EC1V 2NX",5.6
115432,EC1V 2NX,CITY ROAD,185,9248,"KEMP HOUSE 152-160, CITY ROAD, LONDON, EC1V 2NX",2.0
115412,EC1V 2NX,160 CITY ROAD,179,9248,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",1.9
115350,EC1V 2NX,124 CITY ROAD,143,9248,"124 CITY ROAD, LONDON, EC1V 2NX",1.5


In [27]:
MERGES = [
    ("N1 7GU",  "20 WENLOCK ROAD",  "20-22 WENLOCK ROAD"),
    ("EC1V 2NX","124 CITY ROAD",    "124-128 CITY ROAD"),
    ("EC1V 2NX","CITY ROAD",        "152-160 CITY ROAD"),  # this row's FullAddress shows “KEMP HOUSE 152-160…”
    ("EC1V 2NX","160 CITY ROAD",    "152-160 CITY ROAD"),
]

In [28]:
df_uk["Address_street_merged"] = df_uk["Address_street"]
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_163412/863531806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [29]:
df_uk["Address_street_merged"] = df_uk["Address_street"]
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_163412/863531806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [30]:
heavy_100plus_merged.head(10)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
376911,WC2H 9JQ,71-75 SHELTON STREET,11883,12062,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W...",98.5
115386,EC1V 2NX,128 CITY ROAD,5806,9248,"128 CITY ROAD, LONDON, EC1V 2NX",62.8
217131,N1 7GU,20-22 WENLOCK ROAD,5333,5408,"20-22 WENLOCK ROAD, LONDON, N1 7GU",98.6
368254,W1W 7LT,85 GREAT PORTLAND STREET,2409,2843,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",84.7
376098,WC1N 3AX,27 OLD GLOUCESTER STREET,2269,2316,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX",98.0
115354,EC1V 2NX,124 CITY ROAD,2069,9248,"124 CITY ROAD, LONDON, EC1V 2NX",22.4
115953,EC2A 4NE,PAUL STREET,1953,4041,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",48.3
115917,EC2A 4NE,86-90 PAUL STREET,1806,4041,"86-90 PAUL STREET, LONDON, EC2A 4NE",44.7
368079,W1W 5PF,167-169 GREAT PORTLAND STREET,1322,1786,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND...",74.0
376515,WC2A 2JR,7 BELL YARD,1197,1266,"7 BELL YARD, LONDON, WC2A 2JR",94.5


In [31]:
heavy_100plus_merged.query('PostCode_clean == "EC2A 4NE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115953,EC2A 4NE,PAUL STREET,1953,4041,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",48.3
115917,EC2A 4NE,86-90 PAUL STREET,1806,4041,"86-90 PAUL STREET, LONDON, EC2A 4NE",44.7
115901,EC2A 4NE,86 - 90 PAUL STREET,176,4041,"3RD FLOOR, 86 - 90 PAUL STREET, LONDON, EC2A 4NE",4.4


In [32]:
heavy_100plus_merged.query('PostCode_clean == "HA4 7AE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
145701,HA4 7AE,"17 KING EDWARDS ROAD,",585,741,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",78.9
145700,HA4 7AE,17 KING EDWARDS ROAD,123,741,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",16.6


In [35]:
heavy_100plus_merged.query('PostCode_clean == "IP28 7DE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
165239,IP28 7DE,82A JAMES CARTER ROAD,551,2012,"82A JAMES CARTER ROAD, MILDENHALL, IP28 7DE",27.4
165216,IP28 7DE,82 JAMES CARTER ROAD,418,2012,"UNIT A, 82 JAMES CARTER ROAD, MILDENHALL, IP28...",20.8
165330,IP28 7DE,UNIT 82A JAMES CARTER ROAD,335,2012,"UNIT 82A JAMES CARTER ROAD, MILDENHALL, BURY S...",16.7
165334,IP28 7DE,UNIT A 82 JAMES CARTER ROAD,134,2012,"UNIT A 82 JAMES CARTER ROAD, MILDENHALL, BURY ...",6.7
165298,IP28 7DE,SUITE A 82 JAMES CARTER ROAD,129,2012,"SUITE A 82 JAMES CARTER ROAD, MILDENHALL, BURY...",6.4
165275,IP28 7DE,JAMES CARTER ROAD,120,2012,"UNIT A 82, JAMES CARTER ROAD, MILDENHALL, IP28...",6.0


len(postcodes_with_issue), postcodes_with_issue[:10]


Merges need to be added higher up, before we actually create the merged_address

In [38]:
MERGES = [
    # N1 7GU → one row
    ("N1 7GU",  "20 WENLOCK ROAD",            "20-22 WENLOCK ROAD"),

    # EC1V 2NX → keep ONLY two rows: 124–128 and 152–160
    ("EC1V 2NX","124 CITY ROAD",              "124-128 CITY ROAD"),
    ("EC1V 2NX","128 CITY ROAD",              "124-128 CITY ROAD"),
    ("EC1V 2NX","CITY ROAD",                  "152-160 CITY ROAD"),
    ("EC1V 2NX","160 CITY ROAD",              "152-160 CITY ROAD"),

    # EC2A 4NE → one row
    ("EC2A 4NE","PAUL STREET",                "86-90 PAUL STREET"),
    ("EC2A 4NE","86 - 90 PAUL STREET",        "86-90 PAUL STREET"),

    # HA4 7AE → one row
    ("HA4 7AE","17 KING EDWARDS ROAD,",       "17 KING EDWARDS ROAD"),

    # IP28 7DE → one row: canonical "82 JAMES CARTER ROAD"
    ("IP28 7DE","UNIT 82A JAMES CARTER ROAD", "82 JAMES CARTER ROAD"),
    ("IP28 7DE","UNIT A 82 JAMES CARTER ROAD","82 JAMES CARTER ROAD"),
    ("IP28 7DE","SUITE A 82 JAMES CARTER ROAD","82 JAMES CARTER ROAD"),
    ("IP28 7DE","82A JAMES CARTER ROAD",      "82 JAMES CARTER ROAD"),
    ("IP28 7DE","JAMES CARTER ROAD",          "82 JAMES CARTER ROAD"),
]


In [40]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean")[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)
# one representative FullAddress per merged group
fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.dropna().iloc[0] if s.notna().any() else None)
         .reset_index(name="FullAddress_best")
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100).round(1)

heavy_100plus_merged = (heavy_m[heavy_m["Companies_at_Address"] >= 100]
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
                        .rename(columns={"Address_street_merged":"Address_street"}))

In [41]:
heavy_100plus_merged.query('PostCode_clean in ["N1 7GU","EC1V 2NX","EC2A 4NE","HA4 7AE","IP28 7DE"]') \
    .sort_values(["PostCode_clean","Companies_at_Address"], ascending=[True,False])


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115380,EC1V 2NX,124-128 CITY ROAD,8396,9248,"128 CITY ROAD, LONDON, EC1V 2NX",90.8
115405,EC1V 2NX,152-160 CITY ROAD,425,9248,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",4.6
115350,EC1V 2NX,124 CITY ROAD,143,9248,"124 CITY ROAD, LONDON, EC1V 2NX",1.5
115912,EC2A 4NE,86-90 PAUL STREET,3935,4041,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",97.4
145694,HA4 7AE,17 KING EDWARDS ROAD,708,741,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",95.5
165209,IP28 7DE,82 JAMES CARTER ROAD,1687,2012,"UNIT 82A JAMES CARTER ROAD, MILDENHALL, BURY S...",83.8
217119,N1 7GU,20-22 WENLOCK ROAD,5333,5408,"20-22 WENLOCK ROAD, LONDON, N1 7GU",98.6


In [42]:
# Normalize spacing and hyphens on the working key
df_uk["Address_street_merged"] = (
    df_uk["Address_street_merged"]
      .str.replace(r"\s*-\s*", "-", regex=True)   # "124 - 128" -> "124-128"
      .str.replace(r"\s+", " ", regex=True)       # collapse double spaces
      .str.strip()
)

# Make sure EC1V 2NX '124 CITY ROAD' is merged into '124-128 CITY ROAD'
df_uk.loc[
    (df_uk["PostCode_clean"].eq("EC1V 2NX")) &
    (df_uk["Address_street_merged"].str.fullmatch(r"124\s+CITY\s+ROAD")),
    "Address_street_merged"
] = "124-128 CITY ROAD"


/tmp/ipykernel_163412/1296275775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = (


In [43]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

def best_full(s):
    s = s.dropna()
    if s.empty: return None
    m = s.mode()
    return m.iat[0] if not m.empty else s.iloc[0]

fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress":"FullAddress_best"})
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (
    heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100
).round(1)

heavy_100plus_merged = (
    heavy_m[heavy_m["Companies_at_Address"] >= 100]
      .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
      .rename(columns={"Address_street_merged":"Address_street"})
)


In [44]:
heavy_100plus_merged.query('PostCode_clean == "EC1V 2NX"')


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115175,EC1V 2NX,124-128 CITY ROAD,8540,9248,"128 CITY ROAD, LONDON, EC1V 2NX",92.3
115194,EC1V 2NX,152-160 CITY ROAD,459,9248,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",5.0


In [46]:
heavy_100plus_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 376191 to 327750
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         176 non-null    object 
 1   Address_street         176 non-null    object 
 2   Companies_at_Address   176 non-null    int64  
 3   Companies_in_Postcode  176 non-null    int64  
 4   FullAddress_best       176 non-null    object 
 5   Address_share_in_PC    176 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 9.6+ KB


In [62]:
pd.set_option('display.max_rows', 500)
heavy_100plus_merged.sort_values('PostCode_clean')['PostCode_clean'].value_counts().head()

PostCode_clean
BH16 6FA    2
W1W 7LT     2
WA1 1RL     2
EC1V 2NX    2
BD1 3RG     1
Name: count, dtype: int64

In [61]:
heavy_100plus_merged.sort_values('PostCode_clean').head(3)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
6737,B15 2LD,2 WHEELEYS ROAD,138,139,"2 WHEELEYS ROAD, EDGBASTON, BIRMINGHAM, B15 2LD",99.3
7707,B19 2HN,1 GUILDFORD STREET,105,105,"SIX WAYS BUSINESS CENTRE, 1 GUILDFORD STREET, ...",100.0
17515,B90 8AH,LUMANERI HOUSE BLYTHE GATE,101,116,"LUMANERI HOUSE BLYTHE GATE, BLYTHE VALLEY PARK...",87.1


In [63]:
heavy_100plus_merged.query('PostCode_clean == "BH16 6FA"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
28915,BH16 6FA,WAREHAM ROAD,209,529,"LYTCHETT HOUSE 13 FREELAND PARK, WAREHAM ROAD,...",39.5
28905,BH16 6FA,UNIT 13 FREELAND PARK WAREHAM ROAD,128,529,"UNIT 13 FREELAND PARK WAREHAM ROAD, LYTCHETT M...",24.2


In [64]:
heavy_100plus_merged.query('PostCode_clean == "W1W 7LT"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
367570,W1W 7LT,85 GREAT PORTLAND STREET,2409,2843,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",84.7
367584,W1W 7LT,"85 GREAT PORTLAND STREET, FIRST FLOOR",189,2843,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",6.6


In [65]:
heavy_100plus_merged.query('PostCode_clean == "WA1 1RL"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
371803,WA1 1RL,OPTIONIS HOUSE 840 IBIS COURT,172,373,"OPTIONIS HOUSE 840 IBIS COURT, CENTRE PARK, WA...",46.1
371795,WA1 1RL,840 IBIS COURT,155,373,"840 IBIS COURT, CENTRE PARK, WARRINGTON, WA1 1RL",41.6


In [68]:
# Canonicals:
# BH16 6FA  -> "13 FREELAND PARK"
# W1W 7LT   -> "85 GREAT PORTLAND STREET"
# WA1 1RL   -> "840 IBIS COURT"

MERGES += [
    # BH16 6FA (collapse all to "13 FREELAND PARK")
    ("BH16 6FA", "UNIT 13 FREELAND PARK WAREHAM ROAD", "13 FREELAND PARK"),
    ("BH16 6FA", "13 FREELAND PARK WAREHAM ROAD",      "13 FREELAND PARK"),
    ("BH16 6FA", "WAREHAM ROAD",                       "13 FREELAND PARK"),

    # W1W 7LT (drop floor suffix -> canonical)
    ("W1W 7LT",  "85 GREAT PORTLAND STREET, FIRST FLOOR", "85 GREAT PORTLAND STREET"),
    ("W1W 7LT",  "85 GREAT PORTLAND STREET FIRST FLOOR",  "85 GREAT PORTLAND STREET"),  # safety variant

    # WA1 1RL (brand -> base address)
    ("WA1 1RL",  "OPTIONIS HOUSE 840 IBIS COURT", "840 IBIS COURT"),
]


In [69]:
# apply
df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street_merged"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to

# rebuild heavy_m / heavy_100plus_merged (same code you already have)


/tmp/ipykernel_163412/441232984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])


In [71]:
import pandas as pd, numpy as np
from pathlib import Path

# --- 1) Apply your MERGES to the working key
df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])

for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street_merged"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to

# normalize spacing/hyphens (helps catch tiny variants)
df_uk["Address_street_merged"] = (
    df_uk["Address_street_merged"].str.replace(r"\s*-\s*", "-", regex=True)
                                   .str.replace(r"\s+", " ", regex=True)
                                   .str.strip()
)

# --- 2) Recompute heavy table on merged key
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

# one representative FullAddress per group
def best_full(s: pd.Series):
    s = s.dropna()
    if s.empty: return None
    m = s.mode()
    return m.iat[0] if not m.empty else s.iloc[0]

fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress":"FullAddress_best"})
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (
    heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100
).round(1)

# --- 3) Final ≥100 list
heavy_100plus_merged = (
    heavy_m[heavy_m["Companies_at_Address"] >= 100]
      .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
      .rename(columns={"Address_street_merged":"Address_street"})
)

# --- 4) Save
OUT = Path("../raw_data/data_house_full/processed"); OUT.mkdir(parents=True, exist_ok=True)
heavy_m.to_csv(OUT / "heavy_hitters_all_merged.csv", index=False)
heavy_100plus_merged.to_csv(OUT / "heavy_hitters_100plus_merged.csv", index=False)

print(
    f"Unique merged addresses: {len(heavy_m):,} | "
    f'≥100 companies: {len(heavy_100plus_merged):,} | '
    f"Saved to: {OUT}"
)

display(heavy_100plus_merged.head(25))


/tmp/ipykernel_163412/391217320.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])
/tmp/ipykernel_163412/391217320.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = (


Unique merged addresses: 391,857 | ≥100 companies: 173 | Saved to: ../raw_data/data_house_full/processed


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
376181,WC2H 9JQ,71-75 SHELTON STREET,11914,12062,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W...",98.8
115172,EC1V 2NX,124-128 CITY ROAD,8540,9248,"128 CITY ROAD, LONDON, EC1V 2NX",92.3
216704,N1 7GU,20-22 WENLOCK ROAD,5357,5408,"20-22 WENLOCK ROAD, LONDON, N1 7GU",99.1
115689,EC2A 4NE,86-90 PAUL STREET,3940,4041,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",97.5
367563,W1W 7LT,85 GREAT PORTLAND STREET,2617,2843,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",92.1
375377,WC1N 3AX,27 OLD GLOUCESTER STREET,2269,2316,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX",98.0
164885,IP28 7DE,82 JAMES CARTER ROAD,1691,2012,"UNIT 82A JAMES CARTER ROAD, MILDENHALL, BURY S...",84.0
367393,W1W 5PF,167-169 GREAT PORTLAND STREET,1421,1786,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND...",79.6
375791,WC2A 2JR,7 BELL YARD,1197,1266,"7 BELL YARD, LONDON, WC2A 2JR",94.5
371781,WA1 1RG,320 FIRECREST COURT,1119,1125,"320 FIRECREST COURT, CENTRE PARK, WARRINGTON, ...",99.5


In [73]:
heavy_100plus_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173 entries, 376181 to 327743
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         173 non-null    object 
 1   Address_street         173 non-null    object 
 2   Companies_at_Address   173 non-null    int64  
 3   Companies_in_Postcode  173 non-null    int64  
 4   FullAddress_best       173 non-null    object 
 5   Address_share_in_PC    173 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 9.5+ KB
